# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 10 2022 9:11AM,248244,10,MSP214701,"Methapharm, Inc.",Released
1,Oct 10 2022 9:11AM,248244,10,MSP214718,"Methapharm, Inc.",Released
2,Oct 10 2022 9:11AM,248244,10,MSP214728,"Methapharm, Inc.",Released
3,Oct 10 2022 9:10AM,248241,10,MSP214685,"Methapharm, Inc.",Released
4,Oct 10 2022 9:10AM,248241,10,MSP214694,"Methapharm, Inc.",Released
5,Oct 10 2022 9:10AM,248241,10,MSP214700,"Methapharm, Inc.",Released
6,Oct 10 2022 9:10AM,248241,10,MSP214702,"Methapharm, Inc.",Released
7,Oct 10 2022 9:10AM,248241,10,MSP214703,"Methapharm, Inc.",Released
8,Oct 10 2022 9:10AM,248241,10,MSP214706,"Methapharm, Inc.",Released
9,Oct 10 2022 9:10AM,248241,10,MSP214708,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,248238,Released,1
17,248241,Released,29
18,248242,Released,7
19,248243,Released,1
20,248244,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
248238,NaN,NaN,1.0
248241,NaN,NaN,29.0
248242,NaN,NaN,7.0
248243,NaN,NaN,1.0
248244,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
248213,1.0,0.0,22.0
248214,0.0,0.0,1.0
248215,0.0,0.0,1.0
248221,0.0,3.0,25.0
248222,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
248213,1.0,0,22
248214,0.0,0,1
248215,0.0,0,1
248221,0.0,3,25
248222,0.0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,248213,1.0,0,22
1,248214,0.0,0,1
2,248215,0.0,0,1
3,248221,0.0,3,25
4,248222,0.0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,248213,1.0,,22
1,248214,0.0,,1
2,248215,0.0,,1
3,248221,0.0,3,25
4,248222,0.0,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released       int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc."
3,Oct 10 2022 9:10AM,248241,10,"Methapharm, Inc."
32,Oct 10 2022 9:09AM,248243,12,LF of America Corp.
33,Oct 10 2022 9:09AM,248242,10,"Methapharm, Inc."
40,Oct 10 2022 9:04AM,248237,10,ISDIN Corporation
42,Oct 10 2022 9:04AM,248236,10,ISDIN Corporation
104,Oct 10 2022 9:04AM,248233,10,ISDIN Corporation
145,Oct 10 2022 9:04AM,248232,10,ISDIN Corporation
184,Oct 10 2022 9:03AM,248238,19,"Graystone, LLC"
185,Oct 10 2022 8:52AM,248235,10,"CLINUVEL, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc.",0.0,,3
1,Oct 10 2022 9:10AM,248241,10,"Methapharm, Inc.",0.0,,29
2,Oct 10 2022 9:09AM,248243,12,LF of America Corp.,0.0,,1
3,Oct 10 2022 9:09AM,248242,10,"Methapharm, Inc.",0.0,,7
4,Oct 10 2022 9:04AM,248237,10,ISDIN Corporation,0.0,,2
5,Oct 10 2022 9:04AM,248236,10,ISDIN Corporation,0.0,,62
6,Oct 10 2022 9:04AM,248233,10,ISDIN Corporation,0.0,,41
7,Oct 10 2022 9:04AM,248232,10,ISDIN Corporation,0.0,,39
8,Oct 10 2022 9:03AM,248238,19,"Graystone, LLC",0.0,,1
9,Oct 10 2022 8:52AM,248235,10,"CLINUVEL, Inc.",0.0,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc.",3,
1,Oct 10 2022 9:10AM,248241,10,"Methapharm, Inc.",29,
2,Oct 10 2022 9:09AM,248243,12,LF of America Corp.,1,
3,Oct 10 2022 9:09AM,248242,10,"Methapharm, Inc.",7,
4,Oct 10 2022 9:04AM,248237,10,ISDIN Corporation,2,
5,Oct 10 2022 9:04AM,248236,10,ISDIN Corporation,62,
6,Oct 10 2022 9:04AM,248233,10,ISDIN Corporation,41,
7,Oct 10 2022 9:04AM,248232,10,ISDIN Corporation,39,
8,Oct 10 2022 9:03AM,248238,19,"Graystone, LLC",1,
9,Oct 10 2022 8:52AM,248235,10,"CLINUVEL, Inc.",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc.",3,
1,Oct 10 2022 9:10AM,248241,10,"Methapharm, Inc.",29,
2,Oct 10 2022 9:09AM,248243,12,LF of America Corp.,1,
3,Oct 10 2022 9:09AM,248242,10,"Methapharm, Inc.",7,
4,Oct 10 2022 9:04AM,248237,10,ISDIN Corporation,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc.",3,NaN
1,Oct 10 2022 9:10AM,248241,10,"Methapharm, Inc.",29,NaN
2,Oct 10 2022 9:09AM,248243,12,LF of America Corp.,1,NaN
3,Oct 10 2022 9:09AM,248242,10,"Methapharm, Inc.",7,NaN
4,Oct 10 2022 9:04AM,248237,10,ISDIN Corporation,2,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc.",3,0.0
1,Oct 10 2022 9:10AM,248241,10,"Methapharm, Inc.",29,0.0
2,Oct 10 2022 9:09AM,248243,12,LF of America Corp.,1,0.0
3,Oct 10 2022 9:09AM,248242,10,"Methapharm, Inc.",7,0.0
4,Oct 10 2022 9:04AM,248237,10,ISDIN Corporation,2,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3723452,224,0.0
12,248243,1,0.0
16,248225,1,0.0
19,496459,26,3.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3723452,224,0.0
1,12,248243,1,0.0
2,16,248225,1,0.0
3,19,496459,26,3.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,224,0.0
1,12,1,0.0
2,16,1,0.0
3,19,26,3.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,224.0
1,12,Released,1.0
2,16,Released,1.0
3,19,Released,26.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19
Status,,,,
Executing,0.0,0.0,0.0,3.0
Released,224.0,1.0,1.0,26.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19
0,Executing,0.0,0.0,0.0,3.0
1,Released,224.0,1.0,1.0,26.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19
0,Executing,0.0,0.0,0.0,3.0
1,Released,224.0,1.0,1.0,26.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()